# Práctica 2. Resolución de SEL.

**Autor:** Arturo Olivares Martos
***

- **Asignatura:** Métodos Numéricos I.
- **Curso Académico:** 2022-23.
- **Grado:** Doble Grado en Ingeniería Informática y Matemáticas.
- **Grupo:** Único.
- **Profesor:** Lidia Fernández Rodríguez.
- **Descripción:** Práctica 2 de la asignatura Métodos Numéricos I. Resolución de sistemas de ecuaciones lineales (SEL) mediante métodos directos y métodos iterativos.
<!--- **Fecha:** 17 de abril de 2023. -->
<!-- - **Duración:** -->

## Ejercicio 1

Escribe una función que resuelva por sustitución progresiva un sistema en el que la matriz de coeﬁcientes es
triangular inferior. Comprueba que se obtiene el mismo resultado utilizando dicha función que mediante la función
`solve` del módulo `linalg` de Python si lo aplicamos a las matrices siguientes:
$$
    A=\begin{pmatrix}
        1 & 0 & 0 \\
        2 & 2 & 0 \\
        3 & 1 & 5 \\
    \end{pmatrix}
    \qquad
    b=\begin{pmatrix}
        2 & 5 & 1 \\
    \end{pmatrix}.
$$

In [1]:
import numpy as np
import math

In [2]:
A = np.array([[1,0,0], [2,2,0], [3,1,5]])
b = np.array([2,5,1])

In [3]:
def sust_progresiva(A,b):
    
    n = np.size(b)
    x = np.zeros(n)
    
    
    for i in range(0,n):
        x[i] = (b[i]-np.dot(A[i,:i], x[:i])) / A[i,i]
        
    return x

In [4]:
sust_progresiva(A,b)

array([ 2. ,  0.5, -1.1])

In [5]:
np.linalg.solve(A, b)

array([ 2. ,  0.5, -1.1])

## Ejercicio 2
Crea un array `B` de Numpy de 20 ﬁlas y 20 columnas de manera que el elemento $(i, j)$ sea $i + j$ para $1 \leq i,~j \leq 20$.
Ten en cuenta que las ﬁlas y columnas se numeran a partir de 1 pero en Python la numeración empieza en 0.
El primer elemento del array debe ser 2. Crea ahora otro array $A = B + 10\cdot I_{20}$, donde $I_{20}$ representa la matriz
identidad de orden 20.

Crea un vector `b` de 20 elementos todos iguales a 10.

Resuelve el sistema $Ax=b$ usando la función `solve` del módulo `linalg` de Python.

In [6]:
num_fils = 20

A = np.array([[((i+j+2)) for j in range(num_fils)] for i in range(num_fils)]) + 10*np.eye(num_fils)

termino_independiente = 10
b = np.array([termino_independiente for i in range(num_fils)])

In [7]:
np.linalg.solve(A, b)

array([-2.22222222e-01, -2.00000000e-01, -1.77777778e-01, -1.55555556e-01,
       -1.33333333e-01, -1.11111111e-01, -8.88888889e-02, -6.66666667e-02,
       -4.44444444e-02, -2.22222222e-02,  6.22107729e-18,  2.22222222e-02,
        4.44444444e-02,  6.66666667e-02,  8.88888889e-02,  1.11111111e-01,
        1.33333333e-01,  1.55555556e-01,  1.77777778e-01,  2.00000000e-01])

## Ejercicio 3

Modiﬁca el código del método de Gauss de la práctica 1 y haz uno que resuelve un sistema utilizando Gauss con
pivotaje parcial. Utilízalo para resolver el sistema del ejercicio 2.

In [8]:
def sust_regresiva(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante sustitución regresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular superior.
    """
    # A=A.astype(float)     # Con esto se cambian los elementos a tipo flotante pero no es necesario
    # b=b.astype(float) 
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema
                       
    # ahora se resuelve el sistema por sustitución regresiva
    for i in range(n-1,-1,-1):     
        x[i]=(b[i]-np.dot(A[i][i+1:],x[i+1:]))/A[i,i]
                
    return x

def Gauss_Parcial(A,b):
    
    bb=(np.array([b]).transpose()).astype(float) 
    Ab=np.concatenate((A,bb),axis=1)
    n=np.size(b)
    
    # aquí se hacen operaciones por filas en la matriz para conseguir una triangular
    for i in range(n):
        
        # Primero localizamos el máximo del valor absoluto de la columna
        fil_max = np.argmax(abs(A[i:,i]))
        Ab[[i,fil_max+i]] = Ab[[fil_max+i,i]]
        
        for j in range(i+1,n):
            m=Ab[j,i]/Ab[i,i]
            Ab[j]=Ab[j]-m*Ab[i]
            
    # Ahora se resuelve el sistema por sustitución regresiva
    AG=Ab[:,:n]
    bG=Ab[:,n].transpose()
    x=sust_regresiva(AG,bG)
                
    return x

In [10]:
Gauss_Parcial(A,b)

array([-2.22222222e-01, -2.00000000e-01, -1.77777778e-01, -1.55555556e-01,
       -1.33333333e-01, -1.11111111e-01, -8.88888889e-02, -6.66666667e-02,
       -4.44444444e-02, -2.22222222e-02,  2.36217665e-15,  2.22222222e-02,
        4.44444444e-02,  6.66666667e-02,  8.88888889e-02,  1.11111111e-01,
        1.33333333e-01,  1.55555556e-01,  1.77777778e-01,  2.00000000e-01])

## Ejercicio 4

Utilizando la descomposición LU dada en teoría y las sustituciones progresiva y regresiva, resuelve
de nuevo el sistema del ejercicio 2.

In [11]:
def LU_desc(A):
    """
    Esta función calcula la descomposición LU de Doolittle de una matriz A sin intercambio de filas.
    A debe ser un array de Numpy.
    """
    n=np.shape(A)[1]
    U=np.zeros([n,n])
    L=np.eye(n)
    
    for k in range(n-1):
        U[k,k]=A[k,k]-np.dot(L[k,:k],U[:k,k])
        for j in range(k+1,n):
            U[k,j]=A[k,j]-np.dot(L[k,:k],U[:k,j])
        for j in range(k+1):
            L[k+1,j]=(A[k+1,j]-np.dot(L[k+1,:j],U[:j,j]))/U[j,j]
    U[n-1,n-1]=A[n-1,n-1]-np.dot(L[n-1,:n-1],U[:n-1,n-1])
    return L, U

In [12]:
# Ax= b \=>  LUx=b \=> Ly=b.  Ux=y

L, U = LU_desc(A)

y = sust_progresiva(L,b)
x = sust_regresiva (U,y)

In [13]:
x

array([-2.22222222e-01, -2.00000000e-01, -1.77777778e-01, -1.55555556e-01,
       -1.33333333e-01, -1.11111111e-01, -8.88888889e-02, -6.66666667e-02,
       -4.44444444e-02, -2.22222222e-02,  6.34413157e-16,  2.22222222e-02,
        4.44444444e-02,  6.66666667e-02,  8.88888889e-02,  1.11111111e-01,
        1.33333333e-01,  1.55555556e-01,  1.77777778e-01,  2.00000000e-01])

## Ejercicio 5

Deﬁne una función que resuelva un sistema de ecuaciones lineales mediante la factorización de Cholesky. Comprueba
su funcionamiento para intentar resolver el sistema $Ax = b$ donde $b = (2, −5, 1)$ en los casos:
$$
    A_1=\begin{pmatrix}
        2 & 4 & 1\\
        4 & 10 & 1\\
        1 & 1 & 5\\
    \end{pmatrix}
    \qquad
    A_2=\begin{pmatrix}
        2 & 4 & 1\\
        4 & 0 & 1\\
        1 & 1 & 5\\
    \end{pmatrix}
$$

**No** se permite utilizar la función `linalg.cholesky` de `NumPy`. Hay que modiﬁcar el código de la factorización LU
para deﬁnir una función que haga dicha factorización.

In [14]:
def Cholesky_desc(A):
    """
    Esta función calcula la descomposición de Cholesky de una matriz A sin intercambio de filas.
    A debe ser un array de Numpy.
    """
    n=np.shape(A)[1]
    L=np.zeros([n,n])
    
    for k in range(n):
        L[k,k]=math.sqrt(A[k,k]-np.dot(L[k,:k],L[k,:k]))
        for j in range(k+1,n):
            L[j,k]=(A[j,k]-np.dot(L[j,:k],L[k,:k]))/L[k,k]
    return L, L.T

In [15]:
b_5 = np.array([2, -5, 1])

In [16]:
A_5_1 = np.array([[2,4,1], [4,10,1], [1,1,5]])

# Ax= b \=>  LUx=b \=> Ly=b.  Ux=y
L_1, U_1 = Cholesky_desc(A_5_1)

y_1 = sust_progresiva(L_1,b_5)
x_1 = sust_regresiva (U_1,y_1)

In [17]:
x_1

array([11.6875, -5.0625, -1.125 ])

In [18]:
np.linalg.solve(A_5_1, b_5)

array([11.6875, -5.0625, -1.125 ])

In [20]:
A_5_2 = np.array([[2,4,1], [4,0,1], [1,1,5]])

# Ax= b \=>  LUx=b \=> Ly=b.  Ux=y
L_2, U_2 = Cholesky_desc(A_5_2)

y_2 = sust_progresiva(L_2,b_5)
x_2 = sust_regresiva (U_2,y_2)

ValueError: math domain error

En el apartado anterior, podemos ver que al factorizar `A_5_2` no nos deja, ya que su menor principal de orden 2 es negativo, por lo que no es definida positiva y, por tanto, no se le puede aplicar la factorización Cholesky.
El error nos indica que estamos calculando la raíz de un número negativo.

In [21]:
# Ax= b \=>  LUx=b \=> Ly=b.  Ux=y

# Matriz A del ejercicio 2
L_A, U_A = Cholesky_desc(A)

y_A = sust_progresiva(L_A,b)
x_A = sust_regresiva (U_A,y_A)

ValueError: math domain error

In [23]:
np.linalg.det(A)

-8.999999999999942e+21

En este apartado, tampoco podemos calcular la factorización de Cholesky por el mismo motivo. Prueba de esto es que el determinante de la matriz es negativo, como podemos ver en la línea anterior.

## Ejercicio 6

Modiﬁca el código para la resolución de un sistema mediante el método de Jacobi dado en teoría para resolver de
forma aproximada un sistema de ecuaciones lineales utilizando el método de Gauss-Seidel.

En primer lugar me defino estra función auxiliar, que se puede ver su funcionaminto fácilmente.

In [24]:
def TriangularInferior (A):
    """
    Dada una matriz A, devuelve la matriz triangular inferior. Es decir, con 0s por encima de la diagonal principal
    A debe ser un numpyArray
    """
    dim = np.shape(A)
    sup = np.copy(A)
    
    for fil in range(dim[0]):
        for col in range(dim[1]):
            if (fil < col):
                sup[fil][col] = 0
    
    
    return sup

In [25]:
A=np.array([[1,2,3], [4,5,6], [7,8,9]])

In [26]:
TriangularInferior(A)

array([[1, 0, 0],
       [4, 5, 0],
       [7, 8, 9]])

In [27]:
def GaussSeidel(A, b, maxiter=1000, tol=1e-10):
    """
    Esta función resuelve el sistema Ax=b de forma aproximada mediante el método iterativo de Gauss-Seidel empezando las iteraciones
    en el vector nulo.
    Las matrices A y b deben ser arrays de NumPy.
    El número máximo de iteraciones se puede introducir por el usuario y es 1000 por defecto.
    La tolerancia también se puede introducir por el usuario y es 10^(-10) por defecto.
    El método se interrumpe cuando se alcanza el número máximo de iteraciones o cuando la tolerancia es menor que la indicada.
    
    """
    
    n=np.shape(A)[1] # se calcula la dimensión de A
    d=np.diag(A)     # obtenemos un array con los elementos de la diagonal de la matriz A
    x=np.zeros(n)
    y=np.zeros(n)
          
    if np.prod(d)==0:
        print('Hay elementos nulos en la diagonal de la matriz')
    else:
        k=0
        er=1
        nx=1
        while (er>tol*nx and k<maxiter):
            k+=1
            nx=np.linalg.norm(x,1)        
            for i in range(n):
                y[i] = (b[i] - np.dot(A[i,:i],y[:i]) - np.dot(A[i,i+1:],x[i+1:]) )/A[i,i]  
            er=np.linalg.norm(y-x,1)   # calculamos la norma de la diferencia entre y y el resultado de la iteración anterior        
            x=np.copy(y)
           # print('Iteración k=', k, '  x^(k)= ', x ) # descomentar si se quieren ver las iteraciones.
            
        if (k>=maxiter):
            print("Numero máximo de iteraciones alcanzado")
        else:
            print("Calculado en la iteración " + str(k))
        
        return x

In [28]:
def Jacobi(A, b, maxiter=1000, tol=1e-10):
    """
    Esta función resuelve el sistema Ax=b de forma aproximada mediante el método iterativo de Jacobi empezando las iteraciones
    en el vector nulo.
    Las matrices A y b deben ser arrays de NumPy.
    El número máximo de iteraciones se puede introducir por el usuario y es 1000 por defecto.
    La tolerancia también se puede introducir por el usuario y es 10^(-10) por defecto.
    El método se interrumpe cuando se alcanza el número máximo de iteraciones o cuando la tolerancia es menor que la indicada.
    
    """
    
    n=np.shape(A)[1] # se calcula la dimensión de A
    d=np.diag(A)     # obtenemos un array con los elementos de la diagonal de la matriz A
    D=np.diag(d)     # creamos un array diagonal con los elementos de d
    R=A-D            # matriz con los elementos que no están en la diagonal
    x=np.zeros(n)
    y=np.zeros(n)
          
    if np.prod(d)==0:
        print('Hay elementos nulos en la diagonal de la matriz')
    else:
        k=0
        er=1
        nx=1
        while (er>tol*nx and k<maxiter):
            k+=1
            nx=np.linalg.norm(x,1)        
            y=(b - np.dot(R,x)) / d       # aplicamos una iteración del método de Jacobi obteniendo un vector y
            er=np.linalg.norm(y-x,1)      # calculamos la norma de la diferencia entre y y el resultado de la iteración anterior        
            x=y
            #print('Iteración k=', k, '  x^(k)= ', x ) # descomentar si se quieren ver las iteraciones.
        
        if (k>=maxiter):
            print("Numero máximo de iteraciones alcanzado")
        else:
            print("Calculado en la iteración " + str(k))
        return x

In [29]:
A=np.array([[10,4,1],[4,10,1],[1,1,5]])
b=np.array([15,15,7])

In [30]:
np.linalg.solve(A,b)

array([1., 1., 1.])

In [31]:
Jacobi(A,b, 1e4, 1e-12)

Calculado en la iteración 40


array([1., 1., 1.])

In [32]:
GaussSeidel(A,b, 1e4, 1e-12)

Calculado en la iteración 15


array([1., 1., 1.])

## Ejercicio 7

Una vez programado Gauss-Seidel, haz las modiﬁcaciones oportunas para deﬁnir una función que resuelva un sistema utilizando un método de relajación con la constante $\omega$ introducida por el usuario.

In [33]:
def Relajacion(A, b, omega=1, maxiter=1000, tol=1e-10):
    """
    Esta función resuelve el sistema Ax=b de forma aproximada mediante el método iterativo de relajación empezando las iteraciones
    en el vector nulo.
    Las matrices A y b deben ser arrays de NumPy.
    El valor de omega lo aporta el usuario.
    El número máximo de iteraciones se puede introducir por el usuario y es 1000 por defecto.
    La tolerancia también se puede introducir por el usuario y es 10^(-10) por defecto.
    El método se interrumpe cuando se alcanza el número máximo de iteraciones o cuando la tolerancia es menor que la indicada.
    
    """
    
    if (omega <= 0 or omega >= 2):
        print ("El método iterativo no converge para ese valor de omega")
    else:
    
        n=np.shape(A)[1] # se calcula la dimensión de A
        d=np.diag(A)     # obtenemos un array con los elementos de la diagonal de la matriz A
        x=np.zeros(n)
        x_bar=np.zeros(n)
        y=np.zeros(n)

        if np.prod(d)==0:
            print('Hay elementos nulos en la diagonal de la matriz')
        else:
            k=0
            er=1
            nx=1
            while (er>tol*nx and k<maxiter):
                k+=1
                nx=np.linalg.norm(x,1)        
                for i in range(n):
                    x_bar[i] = (b[i] - np.dot(A[i,:i],y[:i]) - np.dot(A[i,i+1:],x[i+1:]) )/A[i,i]
                    y = omega*x_bar + (1-omega)*x
                er=np.linalg.norm(y-x,1)   # calculamos la norma de la diferencia entre y y el resultado de la iteración anterior        
                x=np.copy(y)
               # print('Iteración k=', k, '  x^(k)= ', x ) # descomentar si se quieren ver las iteraciones.


            if (k>=maxiter):
                print("Numero máximo de iteraciones alcanzado")
            else:
                print("Calculado en la iteración " + str(k))
            return x

In [34]:
Relajacion(A,b, 0.8, 1e4, 1e-12)

Calculado en la iteración 30


array([1., 1., 1.])

## Ejericio 8

Deﬁne una matriz $A$ cuadrada de orden 20 cuyo elemento $a_{ij} = \frac{1}{i + j}$ siempre que $i \neq j$ y $a_{ii} = 20\cdot i^2$. Deﬁne también un vector $b$ cuyas componentes sean todas iguales a 20. Resuelve ahora el sistema $Ax = b$ de forma
aproximada mediante los métodos de Jacobi y Gauss-Seidel con un máximo de $10000$ iteraciones y una tolerancia
de $10^{−12}$. Si también has programado el método de relajación, inclúyelo también utilizando $\omega = 0.8$. Comenta los
resultados.


In [35]:
num_fils = 20

A = np.zeros((num_fils, num_fils))
for fil in range(num_fils):
    for col in range(num_fils):
        if (fil==col):
            A[fil,col] = 20*((fil+1)**2)
        else:
            A[fil][col] = 1/(fil+col+2)

termino_independiente = 20
b = np.array([termino_independiente for i in range(num_fils)])

In [36]:
Jacobi(A,b, 1e4, 1e-12)

Calculado en la iteración 8


array([0.99293021, 0.24523958, 0.10932211, 0.06165761, 0.03953924,
       0.02749887, 0.02022665, 0.01550028, 0.0122563 , 0.00993376,
       0.00821401, 0.00690512, 0.00588592, 0.00507681, 0.00442377,
       0.0038891 , 0.00344582, 0.00307423, 0.00275967, 0.00249103])

In [37]:
GaussSeidel(A,b, 1e4, 1e-12)

Calculado en la iteración 5


array([0.99293021, 0.24523958, 0.10932211, 0.06165761, 0.03953924,
       0.02749887, 0.02022665, 0.01550028, 0.0122563 , 0.00993376,
       0.00821401, 0.00690512, 0.00588592, 0.00507681, 0.00442377,
       0.0038891 , 0.00344582, 0.00307423, 0.00275967, 0.00249103])

In [38]:
Relajacion(A,b,0.8, 1e4, 1e-12)

Calculado en la iteración 18


array([0.99293021, 0.24523958, 0.10932211, 0.06165761, 0.03953924,
       0.02749887, 0.02022665, 0.01550028, 0.0122563 , 0.00993376,
       0.00821401, 0.00690512, 0.00588592, 0.00507681, 0.00442377,
       0.0038891 , 0.00344582, 0.00307423, 0.00275967, 0.00249103])

Como se puede ver, los tres métodos convergen, pero el de Gauss-Seidel es el más rápido.

In [39]:
sums = np.array([(A[i][i] - (sum(A[i][:i]) + sum(A[i][i+1:])))  for i in range (num_fils)])

In [40]:
np.min(sums)

17.854641295237272

Como el mínimo es positivo, signitica que la matriz es E.D.D. por filas y, por tanto, Jacobi y Gauss-Seidel convergen.
Además, como omega está entre 0 y 1, también converge relajación.